In [1]:
import os
import argparse
import json
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, ELU
from keras.layers.convolutional import Convolution2D



Using TensorFlow backend.


In [2]:
def get_commaai_model():
    """
    Creates the comma.ai model, and returns a reference to the model
    The comma.ai model's original source code is available at:
    https://github.com/commaai/research/blob/master/train_steering_model.py
    """
    ch, row, col = CH, H, W  # camera format

    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1.,
        input_shape=(row, col, ch),
        output_shape=(row, col, ch)))
    model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode='same', W_regularizer=l2(L2_REG_SCALE)))
    model.add(ELU())
    model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode='same', W_regularizer=l2(L2_REG_SCALE)))
    model.add(ELU())
    model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode='same', W_regularizer=l2(L2_REG_SCALE)))
    model.add(Flatten())
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(512, W_regularizer=l2(0.)))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(1, W_regularizer=l2(0.)))

    model.compile(optimizer=Adam(lr=LR), loss='mean_squared_error')

    return model

In [3]:
import numpy as np
import pandas as pd
from scipy.misc import imread
import matplotlib.pyplot as plt
import pickle

log_file = 'sim_data/driving_log.csv'
df = pd.read_csv(log_file, sep=',',header=None)

print(df.head())

                                                   0   1   2    3    4    5  \
0  /Users/sebasibarguen/Documents/Dev/learn/carnd... NaN NaN  0.0  0.0  0.0   
1  /Users/sebasibarguen/Documents/Dev/learn/carnd... NaN NaN  0.0  0.0  0.0   
2  /Users/sebasibarguen/Documents/Dev/learn/carnd... NaN NaN  0.0  0.0  0.0   
3  /Users/sebasibarguen/Documents/Dev/learn/carnd... NaN NaN  0.0  0.0  0.0   
4  /Users/sebasibarguen/Documents/Dev/learn/carnd... NaN NaN  0.0  0.0  0.0   

          6  
0  0.000007  
1  0.000011  
2  0.000009  
3  0.000013  
4  0.000007  


In [4]:

img_shape = imread(df[0][0]).shape
features = np.zeros([len(df), img_shape[0], img_shape[1], img_shape[2]])
labels = np.zeros([len(df), 1])

for i, row in df.iterrows():
    features[i] = imread(row[0])
    labels[i] = row[3]

# data = {
#     'features': features,
#     'labels': labels
# }

# with open('train.p', 'wr') as f:
#     pickle.dump(data, f)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.3)

In [ ]:
model = get_commaai_model()

model.fit(X_train, y_train)